In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
path = '/data/examples/ibmxfet/'

file_dir = os.listdir(path)

In [3]:
file_dir

['profile.txt',
 'training.csv',
 'testing.csv',
 'behavior_sim.csv',
 'network_sim.csv',
 'usage_data.txt']

In [4]:
profile = pd.read_csv(path+file_dir[0])
training = pd.read_csv(path+file_dir[1])
testing = pd.read_csv(path+file_dir[2])
behavior = pd.read_csv(path+file_dir[3])
network_sim = pd.read_csv(path+file_dir[4])
usage_data = pd.read_csv(path+file_dir[5])

In [5]:
profile.head()

,USER_ID,SRV_TYPE_DESC,SUBSCR_STATUS_CODE,GENDER_CODE,P6M_AVG_NET_INV_AMT,AGE,TENURE,BILL_CITY_NAME,BILL_DISTRICT_NAME,SMS_PROM_IND,IMEI_TYPE,IMEI_MFG_NAME,IMEI_MKT_NAME,IMEI_SMART_OS_FLAG,Credit_IND,AGENCY_FLAG,Third_Party_Payment_IND,FLAT_RATE_4G_IND,P6M_AVG_DATA_USAGE_MB
0,1,4G,A,M,600~800,25~29,66,臺中市,北區,Y,Smart Phone,HTC,10,Android 6.0,N,N,N,Y,4466
1,2,4G,A,F,600~800,75~79,12,新北市,中和,N,Smart Phone,OPPO,R9s,Android 6.0,N,N,N,Y,4681
2,3,4G,A,F,1000~1200,35~39,39,宜蘭縣,五結,Y,Smart Phone,HTC,Desire 830,Android 5.1,Y,Y,Y,Y,4138
3,4,4G,A,M,1200~1400,40~44,21,臺北市,大安區,Y,Smart Phone,Apple,iPhone 6 Plus,iOS 8,Y,N,N,Y,21216
4,6,4G,A,F,1600~1800,25~29,55,嘉義縣,中埔,Y,Smart Phone,Apple,iPhone 6,iOS 8,N,N,N,Y,28655


In [6]:
le = LabelEncoder()

In [7]:
le.fit(profile['P6M_AVG_NET_INV_AMT'])
le.classes_
cost = le.transform(profile['P6M_AVG_NET_INV_AMT'])


In [8]:
le.fit(profile['AGE'])
le.classes_
age = le.transform(profile['AGE'])

In [9]:
tenure = []
for i in range(len(profile)):
    if profile['TENURE'][i] < 12:
        tenure.append(1)
    elif (profile['TENURE'][i]>12)&(profile['TENURE'][i]<36):
        tenure.append(2)
    elif (profile['TENURE'][i]>36)&(profile['TENURE'][i]<60):
        tenure.append(3)
    elif (profile['TENURE'][i]>60)&(profile['TENURE'][i]<120):
        tenure.append(4)
    else:
        tenure.append(5)

In [10]:
le.fit(profile['IMEI_TYPE'])
le.classes_
device = le.transform(profile['IMEI_TYPE'])

In [11]:
le.fit(profile['IMEI_MFG_NAME'])
le.classes_
brand = le.transform(profile['IMEI_MFG_NAME'])

In [12]:
le.fit(profile['IMEI_MKT_NAME'])
le.classes_
brand_model = le.transform(profile['IMEI_MKT_NAME'])

In [13]:
avg_usage = []
for i in range(len(profile)):
    if profile['P6M_AVG_DATA_USAGE_MB'][i] < 2000:
        avg_usage.append(1)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>2000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<5000):
        avg_usage.append(2)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>5000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<8000):
        avg_usage.append(3)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>8000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<10000):
        avg_usage.append(4)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>10000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<12000):
        avg_usage.append(5)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>12000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<15000):
        avg_usage.append(6)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>15000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<20000):
        avg_usage.append(7)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>20000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<30000):
        avg_usage.append(8)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>30000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<50000):
        avg_usage.append(9)
    elif (profile['P6M_AVG_DATA_USAGE_MB'][i]>50000)&(profile['P6M_AVG_DATA_USAGE_MB'][i]<80000):
        avg_usage.append(10)
    else:
        avg_usage.append(11)

In [15]:
gender = pd.get_dummies(profile['GENDER_CODE'])
city = pd.get_dummies(profile['BILL_CITY_NAME'])
sms = pd.get_dummies(profile['SMS_PROM_IND'])
credit = pd.get_dummies(profile['Credit_IND'])
agency = pd.get_dummies(profile['AGENCY_FLAG'])
_3rdpay = pd.get_dummies(profile['Third_Party_Payment_IND'])
_4gfull = pd.get_dummies(profile['FLAT_RATE_4G_IND'])


In [39]:
temp = pd.DataFrame()
append = temp.append([cost,age,tenure,device,brand,brand_model,avg_usage])
append = append.transpose()
append.columns = ['cost','age','tenure','device','brand','brand_model','avg_usage']


In [44]:
encoded_feature = pd.concat([profile['USER_ID'],gender,city,sms,credit,agency,_3rdpay,_4gfull,append],
                            axis=1)
encoded_feature.head()

,USER_ID,F,M,南投縣,嘉義市,嘉義縣,基隆市,宜蘭縣,屏東縣,彰化縣,...,Y,N,Y,cost,age,tenure,device,brand,brand_model,avg_usage
0,1,0,1,0,0,0,0,0,0,0,...,0,0,1,17,1,4,2,7,2,2
1,2,1,0,0,0,0,0,0,0,0,...,0,0,1,17,11,5,2,16,141,2
2,3,1,0,0,0,0,0,1,0,0,...,1,0,1,1,3,3,2,7,34,2
3,4,0,1,0,0,0,0,0,0,0,...,0,0,1,2,4,2,2,2,225,8
4,6,1,0,0,0,1,0,0,0,0,...,0,0,1,4,1,3,2,2,224,8


In [47]:
usage_sum = usage_data.groupby('USER_ID',as_index=False).sum()
usage_sum.head()

,USER_ID,MO_CALL_DUR,MT_CALL_DUR,DATA_USG_MB
0,1,2412,6640,2787.69
1,2,741,866,3147.82
2,3,393,51,5376.46
3,4,3207,6616,12812.16
4,6,769,67,29411.16


In [48]:
with open('usage_sum','wb') as file:
    pickle.dump(usage_sum,file)

In [50]:
call_out_usage = []
for i in range(len(usage_sum)):
    if usage_sum['MO_CALL_DUR'][i] < 10:
        call_out_usage.append(1)
    elif (usage_sum['MO_CALL_DUR'][i]>10)&(usage_sum['MO_CALL_DUR'][i]<60):
        call_out_usage.append(2)
    elif (usage_sum['MO_CALL_DUR'][i]>60)&(usage_sum['MO_CALL_DUR'][i]<180):
        call_out_usage.append(3)
    elif (usage_sum['MO_CALL_DUR'][i]>180)&(usage_sum['MO_CALL_DUR'][i]<300):
        call_out_usage.append(4)
    elif (usage_sum['MO_CALL_DUR'][i]>300)&(usage_sum['MO_CALL_DUR'][i]<500):
        call_out_usage.append(5)
    elif (usage_sum['MO_CALL_DUR'][i]>500)&(usage_sum['MO_CALL_DUR'][i]<750):
        call_out_usage.append(6)
    elif (usage_sum['MO_CALL_DUR'][i]>750)&(usage_sum['MO_CALL_DUR'][i]<1000):
        call_out_usage.append(7)
    elif (usage_sum['MO_CALL_DUR'][i]>1000)&(usage_sum['MO_CALL_DUR'][i]<1400):
        call_out_usage.append(8)
    elif (usage_sum['MO_CALL_DUR'][i]>1400)&(usage_sum['MO_CALL_DUR'][i]<2000):
        call_out_usage.append(9)
    elif (usage_sum['MO_CALL_DUR'][i]>2000)&(usage_sum['MO_CALL_DUR'][i]<4000):
        call_out_usage.append(10)
    else:
        call_out_usage.append(11)

In [52]:
call_in_usage = []
for i in range(len(usage_sum)):
    if usage_sum['MT_CALL_DUR'][i] < 20:
        call_in_usage.append(1)
    elif (usage_sum['MT_CALL_DUR'][i]>20)&(usage_sum['MT_CALL_DUR'][i]<90):
        call_in_usage.append(2)
    elif (usage_sum['MT_CALL_DUR'][i]>90)&(usage_sum['MT_CALL_DUR'][i]<180):
        call_in_usage.append(3)
    elif (usage_sum['MT_CALL_DUR'][i]>180)&(usage_sum['MT_CALL_DUR'][i]<320):
        call_in_usage.append(4)
    elif (usage_sum['MT_CALL_DUR'][i]>320)&(usage_sum['MT_CALL_DUR'][i]<550):
        call_in_usage.append(5)
    elif (usage_sum['MT_CALL_DUR'][i]>550)&(usage_sum['MT_CALL_DUR'][i]<800):
        call_in_usage.append(6)
    elif (usage_sum['MT_CALL_DUR'][i]>800)&(usage_sum['MT_CALL_DUR'][i]<1100):
        call_in_usage.append(7)
    elif (usage_sum['MT_CALL_DUR'][i]>1100)&(usage_sum['MT_CALL_DUR'][i]<1600):
        call_in_usage.append(8)
    elif (usage_sum['MT_CALL_DUR'][i]>1600)&(usage_sum['MT_CALL_DUR'][i]<2400):
        call_in_usage.append(9)
    elif (usage_sum['MT_CALL_DUR'][i]>2400)&(usage_sum['MT_CALL_DUR'][i]<4000):
        call_in_usage.append(10)
    else:
        call_in_usage.append(11)

In [53]:
data_usage = []
for i in range(len(usage_sum)):
    if usage_sum['DATA_USG_MB'][i] < 400:
        data_usage.append(1)
    elif (usage_sum['DATA_USG_MB'][i]>400)&(usage_sum['DATA_USG_MB'][i]<1000):
        data_usage.append(2)
    elif (usage_sum['DATA_USG_MB'][i]>1000)&(usage_sum['DATA_USG_MB'][i]<2000):
        data_usage.append(3)
    elif (usage_sum['DATA_USG_MB'][i]>2000)&(usage_sum['DATA_USG_MB'][i]<3200):
        data_usage.append(4)
    elif (usage_sum['DATA_USG_MB'][i]>3200)&(usage_sum['DATA_USG_MB'][i]<5000):
        data_usage.append(5)
    elif (usage_sum['DATA_USG_MB'][i]>5000)&(usage_sum['DATA_USG_MB'][i]<7200):
        data_usage.append(6)
    elif (usage_sum['DATA_USG_MB'][i]>7200)&(usage_sum['DATA_USG_MB'][i]<10000):
        data_usage.append(7)
    elif (usage_sum['DATA_USG_MB'][i]>10000)&(usage_sum['DATA_USG_MB'][i]<14000):
        data_usage.append(8)
    elif (usage_sum['DATA_USG_MB'][i]>14000)&(usage_sum['DATA_USG_MB'][i]<24000):
        data_usage.append(9)
    else:
        data_usage.append(10)

In [54]:
temp = pd.DataFrame()
append = temp.append([call_out_usage,call_in_usage,data_usage])
append = append.transpose()
append.columns = ['call_out_usage','call_in_usage','data_usage']


In [56]:
encoded_feature = pd.concat([encoded_feature,append],axis=1)
encoded_feature.head()

,USER_ID,F,M,南投縣,嘉義市,嘉義縣,基隆市,宜蘭縣,屏東縣,彰化縣,...,cost,age,tenure,device,brand,brand_model,avg_usage,call_out_usage,call_in_usage,data_usage
0,1,0,1,0,0,0,0,0,0,0,...,17,1,4,2,7,2,2,10,11,4
1,2,1,0,0,0,0,0,0,0,0,...,17,11,5,2,16,141,2,6,7,4
2,3,1,0,0,0,0,0,1,0,0,...,1,3,3,2,7,34,2,5,2,6
3,4,0,1,0,0,0,0,0,0,0,...,2,4,2,2,2,225,8,10,11,8
4,6,1,0,0,0,1,0,0,0,0,...,4,1,3,2,2,224,8,7,2,10


In [57]:
lis = []
for name in range(len(encoded_feature.columns)):
    d = str(encoded_feature.columns[name]).replace('<','')
    lis.append(d)

In [58]:
encoded_feature.columns = lis

In [59]:
encoded_feature.head()

,USER_ID,F,M,南投縣,嘉義市,嘉義縣,基隆市,宜蘭縣,屏東縣,彰化縣,...,cost,age,tenure,device,brand,brand_model,avg_usage,call_out_usage,call_in_usage,data_usage
0,1,0,1,0,0,0,0,0,0,0,...,17,1,4,2,7,2,2,10,11,4
1,2,1,0,0,0,0,0,0,0,0,...,17,11,5,2,16,141,2,6,7,4
2,3,1,0,0,0,0,0,1,0,0,...,1,3,3,2,7,34,2,5,2,6
3,4,0,1,0,0,0,0,0,0,0,...,2,4,2,2,2,225,8,10,11,8
4,6,1,0,0,0,1,0,0,0,0,...,4,1,3,2,2,224,8,7,2,10


In [60]:
training[training['USER_ID']==1]

,USER_ID,date,y
0,1,2018-03-03,0.488242
1,1,2018-03-04,0.481169
2,1,2018-03-05,0.015337
3,1,2018-03-06,0.022330
4,1,2018-03-07,0.019055
5,1,2018-03-08,0.017952
6,1,2018-03-09,0.019987
7,1,2018-03-10,0.431245
8,1,2018-03-11,0.046564
9,1,2018-03-12,0.460840


In [61]:
d = {}
for i in range(3001):
    d[i] = pd.DataFrame(training[training['USER_ID']==i])

In [62]:
for key in d:
    d[key] = d[key].reset_index()

In [63]:
df = pd.concat([d[key] for key in d], axis=1)
df = df.drop(['index','USER_ID'],axis=1)
df = df.dropna(axis=1)
df['DATE'] = df.iloc[:,0]
df = df.drop('date', axis=1)
df = df.transpose()
df.index = range(2629)
df.columns = df.iloc[2628]
df = df.reindex(df.index.drop(2628))
df.index = range(2628)

In [ ]:
# # weekday: 週一~週五 weekend: 週六~週日
# weekday1 = df.iloc[:,2:7]
# weekday2 = df.iloc[:,9:13]
# weekday = pd.concat([weekday1,weekday2],axis=1)

# weekend1 = df.iloc[:,:2]
# weekend2 = df.iloc[:,7:9]
# weekend = pd.concat([weekend1, weekend2],axis=1)

# weekday.columns = ['2018-03-05','2018-03-06','2018-03-07','2018-03-08','2018-03-09','2018-03-12','2018-03-13','2018-03-14','2018-03-15']
# weekend.columns = ['2018-03-03','2018-03-04','2018-03-10','2018-03-11']

In [64]:
# weekday: 週一~週四 weekend: 週五~週日
weekday1 = df.iloc[:,2:6]
weekday2 = df.iloc[:,9:13]
weekday = pd.concat([weekday1,weekday2],axis=1)

weekend1 = df.iloc[:,:2]
weekend2 = df.iloc[:,6:9]
weekend = pd.concat([weekend1, weekend2],axis=1)

weekday.columns = ['2018-03-05','2018-03-06','2018-03-07','2018-03-08','2018-03-12','2018-03-13','2018-03-14','2018-03-15']
weekend.columns = ['2018-03-03','2018-03-04','2018-03-09','2018-03-10','2018-03-11']

In [65]:
weekend['med'] = weekend.median(axis=1)
weekday['med'] = weekday.median(axis=1)

In [67]:
encoded_feature.to_csv('encoded_feature.csv',index=None)
weekday.to_csv('weekday.csv',index=None)
weekend.to_csv('weekend.csv',index=None)
df.to_csv('date.csv',index=None)